---
Author: Jeapo
title: Smartphone - Based Sensing of Pendulum Vibrations
jupyter: python3
format: 
    html: 
        toc: true
        number-sections: true
        html-math-method: katex
        toc-expand: 3
    pdf: 
        toc: true
        number-sections: true
        colorlinks: true

crossref:
  chapters: true
editor: 
    render-on-save: true
---

**Instructor:** Mohammad Talebi-Kalaleh  
**Course:** Sensing Techniques and Data Analytics — Fall 2025

**Student ID:** `STUDENT_ID`  
**Name:** Your Name Here  
**Date:** YYYY-MM-DD  

**Tool Chains**: `Jupyter`->`Quarto`->`PDF`



# Background
## Assignment Outline

1. Setup & Imports  
2. Utility functions (data loading, peak detection, damping fit, stats)  
3. Part A — Short Pendulum (L ≈ 0.3 m)  
4. Part B — Long Pendulum (L ≈ 1.0 m)  
5. Part C — Nonlinear Regime (large angles)  
6. Part D — Noise discussion  
7. Figures & Data saving helpers  
8. Submission checklist

Each analysis part includes: data loading, visualization, statistical descriptors, period estimation, damping estimation, and discussion cells.

In [ ]:
#| label: Imports and settings
import os
import glob
import numpy as np
import pandas as pd
import scipy.signal as signal
import scipy.optimize as optimize
from scipy.stats import skew, kurtosis
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

# Ensure figures render in notebook
%matplotlib inline

# Reproducible plotting size policy
plt.rcParams['figure.figsize'] = (10,4)

# Paths (edit STUDENT_ID below)
STUDENT_ID = "STUDENT_ID"
BASE_DIR = f"."
DATA_DIR = os.path.join(BASE_DIR, 'Data')
FIG_DIR = os.path.join(BASE_DIR, 'Figures')

## Experiment Preparation

### Materials
1. Smartphone:
    Type: Xiaomi 10
    Target sensor: accelerator
2. String
3. Toilet paper roll
4. Camera: Another smartphone
5. Softer & Version: PhyPhox 1.2.0
6. Cross beam: 1 Chopstick
6. Angle measurement: Compass App of Smartphone

### Structure

The pendulum device was constructed as follows:

- A chopstick was used as the cross beam and fixed horizontally to serve as the support.
- Four lengths of string were cut and tied to the chopstick, spaced evenly to ensure stability.
- The lower ends of the strings were attached to a toilet paper roll, which acted as the pendulum bob holder.
- A smartphone (Xiaomi 10) was placed securely inside the toilet paper roll, with its main axis (y) aligned along the swing direction.
- The PhyPhox app (version 1.2.0) was installed on the smartphone to record acceleration data.
- A second smartphone was used as a camera to record the motion for reference and angle measurement.
- The initial angle of displacement was measured using the compass app on the smartphone before release.

This setup allowed for accurate measurement of pendulum motion using the smartphone's accelerometer, while the camera provided visual confirmation and angle calibration.

## Theoretical Model

### Small-Angle Pendulum Period

For small angular displacements ($\theta \le 15^\circ$), the period of a simple pendulum is given by @eq-simple-pendulum:
$$
T_{n}=2\pi\sqrt{\frac{L}{g}}
$${#eq-simple-pendulum}

where:

  * $T\_n$ = oscillation period (seconds)
  * $L$ = pendulum length (meters)
  * $g$ = gravitational acceleration ($9.81, m/s^2$)

In [ ]:
#| label: Theoretical data for comparison
def generate_theoretical_pendulum(L, t_max=10, dt=0.01, theta0=15, g=9.81):
    """Generate theoretical pendulum motion for small angles
    Args:
        L (float): pendulum length in meters
        t_max (float): simulation duration in seconds
        dt (float): time step in seconds
        theta0 (float): initial angle in radians
        g (float): gravitational acceleration (m/s^2)
    Returns:
        t, theta: time and angle arrays
    """
    t = np.arange(0, t_max, dt)
    omega = np.sqrt(g/L)  # natural frequency
    theta = theta0 * np.cos(omega * t)  # solution for small angles
    return t, theta

def theoretical_period(L, g=9.81):
    """Calculate theoretical period of a simple pendulum"""
    return 2 * np.pi * np.sqrt(L / g)

# Generate and plot theoretical data for both pendulum lengths
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 4))

# Short pendulum
L_short = 0.3 # part A
t_short, theta_short = generate_theoretical_pendulum(L_short)
T_short = theoretical_period(L_short)
ax1.plot(t_short, theta_short, 'b-', label=f'Theory (T={T_short:.3f}s)')
ax1.set_title(f'Short Pendulum (L={L_short}m)')
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Angle (rad)')
ax1.grid(True)
ax1.legend()

# Long pendulum
L_long = 1.0 # part B
t_long, theta_long = generate_theoretical_pendulum(L_long)
T_long = theoretical_period(L_long)
ax2.plot(t_long, theta_long, 'r-', label=f'Theory (T={T_long:.3f}s)')
ax2.set_title(f'Long Pendulum (L={L_long}m)')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Angle (rad)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()
print(f'Theoretical periods:\nShort pendulum: {T_short:.3f}s\nLong pendulum: {T_long:.3f}s')

### Damped Oscillation Model

The amplitude envelope of a damped pendulum follows exponential decay, as shown in @eq-damped-pendulum:
$$
A(t)=A_{0} \exp(-\zeta\frac{2\pi}{T_{n}}t)
$$ {#eq-damped-pendulum}

where:

  * $A(t)$ = amplitude at time t
  * $A_0$ = initial amplitude
  * $\zeta$ = damping ratio (dimensionless)
  * $t$ = elapsed time (seconds)

In [ ]:
#| label: Damped Oscillation
def generate_damped_pendulum(L, t_max=10, dt=0.01, theta0=15, zeta=0.1, g=9.81):
    """Generate damped pendulum motion
    Args:
        L (float): pendulum length in meters
        t_max (float): simulation duration in seconds
        dt (float): time step in seconds
        theta0 (float): initial angle in radians
        zeta (float): damping ratio (dimensionless)
        g (float): gravitational acceleration (m/s^2)
    Returns:
        t, theta: time and angle arrays
    """
    t = np.arange(0, t_max, dt)
    omega = np.sqrt(g/L)  # natural frequency
    # Damped solution
    theta = theta0 * np.exp(-zeta * omega * t) * np.cos(omega * t)
    return t, theta

# Plot comparison of ideal vs damped oscillation
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6))

# Short pendulum comparison
t_short_damped, theta_short_damped = generate_damped_pendulum(L_short, zeta=0.1)
ax1.plot(t_short, theta_short, 'b-', label='Ideal', alpha=0.7)
ax1.plot(t_short_damped, theta_short_damped, 'r--', label='Damped (ζ=0.1)')
ax1.set_title(f'Short Pendulum (L={L_short}m)')
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Angle (rad)')
ax1.grid(True)

# Long pendulum comparison
t_long_damped, theta_long_damped = generate_damped_pendulum(L_long, zeta=0.1)
ax2.plot(t_long, theta_long, 'b-', label='Ideal', alpha=0.7)
ax2.plot(t_long_damped, theta_long_damped, 'r--', label='Damped (ζ=0.1)')
ax2.set_title(f'Long Pendulum (L={L_long}m)')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Angle (rad)')
ax2.grid(True)

# Add the +- A_0 exp(-zeta 2pi/T_n t) lines
for ax, t, theta0, Tn in [
    (ax1, t_short_damped, 15, theoretical_period(L_short)),
    (ax2, t_long_damped, 15, theoretical_period(L_long))
]:
    envelope = theta0 * np.exp(-0.1 * 2 * np.pi / Tn * t)
    ax.plot(t, envelope, 'k:', label=r'$+A_0 e^{-\zeta 2\pi/T_n t}$')
    ax.plot(t, -envelope, 'k:', label=r'$-A_0 e^{-\zeta 2\pi/T_n t}$')

ax1.legend(loc='upper right')
ax2.legend(loc='upper right')

plt.tight_layout()
plt.show()

# Data Preprocessing

In [ ]:
#| label: extract and rename data
#| echo: false
import zipfile
import os
import glob
import pandas as pd

def extract_raw_data_from_zip_files(folder_path, experiment):
    """
    Finds all zip files in a folder, extracts 'Raw Data.csv' from each,
    and saves it with the zip file's name in a new 'extracted_data' folder.

    Args:
        folder_path (str): The path to the folder containing zip files.
    """
    # Define the name of the CSV file and the output directory
    csv_filename = "Raw Data.csv"
    output_dir = os.path.join(folder_path,"")

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created output directory: {output_dir}")

    # Find all zip files in the specified folder
    zip_files = glob.glob(os.path.join(output_dir,experiment, "*.zip"))
    
    if not zip_files:
        print("No zip files found in the specified folder. Exiting.")
        return

    print(f"Found {len(zip_files)} zip files to process.")

    for zip_filepath in zip_files:
        try:
            with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
                # Get the base name of the zip file without the extension
                base_name = os.path.splitext(os.path.basename(zip_filepath))[0]
                
                # Find the path to the CSV file inside the zip, accounting for nesting
                csv_member_path = None
                for member in zip_ref.namelist():
                    if member.endswith(csv_filename):
                        csv_member_path = member
                        break
                
                if not csv_member_path:
                    print(f"Warning: '{csv_filename}' not found in '{os.path.basename(zip_filepath)}'. Skipping.")
                    continue
                
                # Extract the file to a temporary location
                temp_extract_dir = os.path.join(output_dir, base_name)
                zip_ref.extract(csv_member_path, temp_extract_dir)
                
                # The extracted file's path
                extracted_file_path = os.path.join(temp_extract_dir, csv_member_path)

                # Define the new filename and path
                new_csv_filename = f"{base_name}.csv"
                new_csv_path = os.path.join(output_dir, new_csv_filename)
                
                # Move and rename the extracted file
                os.rename(extracted_file_path, new_csv_path)

                # Clean up the temporary directory structure
                os.rmdir(os.path.dirname(extracted_file_path))

                print(f"Successfully extracted and renamed '{os.path.basename(zip_filepath)}' to '{new_csv_filename}'.")

        except (zipfile.BadZipFile, FileNotFoundError) as e:
            print(f"Error processing '{os.path.basename(zip_filepath)}': {e}")
        except Exception as e:
            print(f"An unexpected error occurred with '{os.path.basename(zip_filepath)}': {e}")

if __name__ == "__main__" and False:
    folder_path = "./Data"
    experiment_type = "/Raw/partC"
    
    # Run the function to process the files
    extract_raw_data_from_zip_files(folder_path, experiment=experiment_type)

In [ ]:
#| label: load_trial_csv

def load_trial_csv(path,start=500,end=10000):
    """Load CSV exported from PhyPhox. 
    Assumes time column 'time' (s) and acceleration axis 'accel' aligned with swing direction.
    Adjust column names below if your CSV uses different headers."""
    df = pd.read_csv(path)
    time_col = df.columns[0]
    # choose accel axis heuristically
    accel_col = df.columns[2]
    df = df.rename(columns={time_col: 'time', accel_col: 'accel'})
    return df[['time','accel']].iloc[start:end]  # trim start/end

In [ ]:
#| label: compute descriptors
#| echo: false
#| output: false
def compute_descriptors(x):
    return {
        'Mean (µ)': np.mean(x),
        'Variance (σ²)': np.var(x, ddof=1),
        'Root Mean Square (RMS)': np.sqrt(np.mean(np.square(x))),
        'Skewness (γ₁)': skew(x),
        'Kurtosis (γ₂)': kurtosis(x, fisher=False)
    }

## Peak-detection algorithms

The estimate_period_via_peaks function is designed to estimate the periods of oscillatory signals by identifying the time intervals between successive peaks in the data. It uses the find_peaks method from the scipy.signal module to locate prominent peaks in the signal, then calculates the time differences between these peaks to determine the periods. This approach is commonly used for analyzing periodic signals, such as those from pendulum or vibration experiments.

Key points:

Inputs:
time: Array of time values.
signal_data: Array of signal measurements (e.g., acceleration).
prominence and distance: Parameters to control peak detection sensitivity.
Outputs:
periods: Array of estimated periods (time differences between peaks).
peaks: Indices of detected peaks.
This function is useful for extracting the fundamental period of a signal in experimental data analysis.

In [ ]:
#| label: estimate period via peaks
def estimate_period_via_peaks(time, signal_data, prominence=0.5, distance=None):
    # find peaks (positive swings)
    peaks, props = signal.find_peaks(signal_data, prominence=prominence, distance=distance)
    if len(peaks) < 2:
        return np.nan, peaks
    peak_times = time[peaks]
    periods = np.diff(peak_times)
    return periods, peaks

In [ ]:
#| label: fit exponential decay
def fit_exponential_decay(peak_times, peak_amps, Tn=None):
    """Fit A(t) = A0 * exp(-zeta * 2*pi / Tn * t)
    If Tn is None, fit exponent parameter directly as A0 * exp(-k t) and compute zeta from k and Tn supplied per-trial.
    Returns dict with A0, zeta (if possible), k (decay rate), covariance.
    """
    # fit ln(A) = ln(A0) - k t  where k = zeta * 2*pi / Tn
    positive_mask = peak_amps > 0
    t = np.array(peak_times)[positive_mask]
    A = np.array(peak_amps)[positive_mask]
    if len(A) < 2:
        return {'A0': np.nan, 'k': np.nan, 'zeta': np.nan}
    logA = np.log(A)
    coef = np.polyfit(t, logA, 1)
    k = -coef[0]  # because logA = ln(A0) - k t
    A0 = np.exp(coef[1])
    zeta = np.nan
    if Tn is not None and Tn>0:
        zeta = k * Tn / (2*np.pi)
    return {'A0': A0, 'k': k, 'zeta': zeta}

In [ ]:
#| label: theoretical period
def theoretical_period(L, g=9.81):
    return 2*np.pi*np.sqrt(L/g)

# Part A: Short Pendulum (L ≈ 0.3 m)

**Instructions:** Place CSV files for Part A into `HW1_STUDENTID/Data/` with names like `partA_trial01.csv` ... `partA_trial10.csv`. The code cells below will loop through trials, compute descriptors, estimate period, fit damping, and summarize results.

## Data Collection

In [ ]:
#| echo: false
file_pattern = os.path.join(DATA_DIR, 'partA_trial*.csv')
files = sorted(glob.glob(file_pattern))

## Statistical Analysis

For part A, I collect 11 trails, For each trial, compute:
• Mean (µ)
• Variance (σ2)
• Root Mean Square (RMS)
• Skewness (γ1)
• Kurtosis (γ2)

List all Part A trials and compute descriptors, output as @tbl-part-a, for assuring the data consistence, exclude the begining and the ending data, only keep the middle (from t=500 to t=10000)

In [ ]:
#| label: tbl-part-a
#| tbl-cap: Summary of Part A
#| output: true
#| echo: false
summary = []

for fpath in files:
    df = load_trial_csv(fpath)
    x = df['accel'].values
    desc = compute_descriptors(x)
    summary.append({
        'file': os.path.basename(fpath),
        **desc
    })

summary = pd.DataFrame(summary)
summary

## Period Estimation
> * Apply autocorrelation or peak-detection algorithms
> * Report mean period and standard deviation across trials: $\overline{T_n} = T_n \pm \sigma_T$
> * Compare with theoretical prediction (@eq-simple-pendulum)

Throught these below code, we candetect the peak and We can obtain the time difference between two points before and after. We already get the each period, and then just need compute the mean period and standard deviation as shown in @tbl-mean-period-std-a.

In [ ]:
#| label: fig-overview-a
#| fig-cap: Data overview of part A
#| echo: false
L_short = 0.3  # meters (adjust if measured)
# print(f"Found {len(files)} files for Part A analysis.")
results = []

# Create a 2x5 subplot grid for all trials
fig, axes = plt.subplots(2, 5, figsize=(20, 8))
axes = axes.ravel()  # flatten axes array for easier indexing

for idx, fpath in enumerate(files):
    if idx >= 10:  # limit to 10 figures
        break

    df_A = load_trial_csv(fpath)
    t = df_A['time'].values
    x = df_A['accel'].values
    
    # Plot in the corresponding subplot
    ax = axes[idx]
    ax.plot(t, x, 'b-', alpha=0.7)
    ax.set_title(f'Trial {idx+1}: {os.path.basename(fpath)}')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Accel (units)')
    ax.grid(True)
plt.show()

Compute the mean period and standard deviation as shown in @tbl-mean-period-std-a.

In [ ]:
#| label: tbl-mean-period-std-a
#| tbl-cap: Part A analysis loop (short pendulum)
#| echo: false
for idx, fpath in enumerate(files):
    # Continue with analysis (not shown in plot)
    x_smooth = signal.savgol_filter(x, 11, 3)
    periods, peaks = estimate_period_via_peaks(t, x_smooth, prominence=np.std(x_smooth)*0.5)
    if isinstance(periods, np.ndarray):
        mean_period = np.mean(periods)
        std_period = np.std(periods, ddof=1)
    else:
        mean_period = np.nan
        std_period = np.nan
    # peak amplitudes and times
    peak_times = t[peaks] if len(peaks)>0 else np.array([])
    peak_amps = np.abs(x_smooth[peaks]) if len(peaks)>0 else np.array([])
    decay = fit_exponential_decay(peak_times, peak_amps, Tn=mean_period)
    result = {
        'file': os.path.basename(fpath),
        'mean_period': mean_period,
        'std_period': std_period,
        'A0': decay['A0'],
        'k': decay['k'],
        'zeta': decay['zeta'],
        **desc
    }
    results.append(result)

results_df = pd.DataFrame(results)
results_df

The theoretical can easily compute as following via @eq-simple-pendulum:
$$
\begin{align}
T_{n}&=2\pi\sqrt{\frac{L}{g}} \\
&= 2\pi\sqrt{\frac{0.32}{9.81}} \\
&= 1.135 (s)
\end{align}
$$

In [ ]:
#| echo: false
#| label: compare-part-a

theoretical_period_a = 1.1348
mean_period_a = float(np.round(results_df['mean_period'].mean(),4))
std_period_a = float(np.round(results_df['mean_period'].std(ddof=1),4))
diff_period_a = float(np.round(theoretical_period_a-mean_period_a,4))
# print(f"Mean period: {mean_period:.4f} s, Std: {std_period:.4f} s")

The 10 trials mean period is `{python} mean_period_a` s, and the standard deviation of mean period is `{python} std_period_a` s.

Compare with theoretical, the diffirence is `{python} diff_period_a`. 


## Damping Analysis
Using the last trial as example, We can use images to further reveal the characteristics of the data. @fig-simple-pendulum-analysis-1

In [ ]:
#| label: fig-simple-pendulum-analysis
#| fig-cap: Create subplots for comparison
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Raw vs Smoothed Signal
ax1.plot(t, x, 'b-', alpha=0.5, label='Raw')
ax1.plot(t, x_smooth, 'r-', alpha=0.5, label='Smoothed')
ax1.set_title('Signal Comparison')
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Acceleration')
ax1.grid(True)
ax1.legend()

# Plot 2: Peak Detection
ax2.plot(t, x_smooth, 'b-')
if len(peaks) > 0:
    ax2.plot(t[peaks], x_smooth[peaks], 'ro', label='Peaks')
ax2.set_title('Peak Detection')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Acceleration')
ax2.grid(True)
ax2.legend()

# Plot 3: Period Analysis
if isinstance(periods, np.ndarray) and len(periods) > 0:
    ax3.plot(periods, 'g.-')
    ax3.axhline(y=np.mean(periods), color='r', linestyle='--', 
                label=f'Mean: {np.mean(periods):.4f}s')
    ax3.set_title('Period Variation')
    ax3.set_xlabel('Peak Index')
    ax3.set_ylabel('Period (s)')
    ax3.grid(True)
    ax3.legend()

# Plot 4: Peak Amplitude Decay
if len(peak_times) > 0:
    ax4.plot(peak_times, peak_amps, 'b.', label='Peak Amplitudes')
    ax4.set_title('Amplitude Decay')
    ax4.set_xlabel('Time (s)')
    ax4.set_ylabel('Peak Amplitude')
    ax4.grid(True)
    ax4.legend()

plt.tight_layout()
plt.show()

**Important Tip:** We cannon get the pricise amplitudes, because the values we recorded are accelerator number.
TODO: mybe we can use the accelerator inversely calculate the amplitude, but this exce the scope of this assignment.

In [ ]:
#| label: fig-peak-amplitudes-part-a
#| fig-cap: Peak amplitudes over time for 10 trials (Part A)
fig, ax = plt.subplots(figsize=(12, 6))
for idx, fpath in enumerate(files[:10]):
    df = load_trial_csv(fpath)
    t = df['time'].values
    x = df['accel'].values
    x_smooth = signal.savgol_filter(x, 11, 3)
    periods, peaks = estimate_period_via_peaks(t, x_smooth, prominence=np.std(x_smooth)*0.5)
    if len(peaks) > 0:
        peak_times = t[peaks]
        peak_amps = np.abs(x_smooth[peaks])
        ax.plot(peak_times, peak_amps/0.31656, marker='o', linestyle='-', label=f'Trial {idx+1}', alpha=0.7)

ax.set_xlabel('Time (s)')
ax.set_ylabel('Peak Amplitude')
ax.set_title('Peak Amplitudes Over Time (10 Trials, Part A)')
ax.grid(True)
ax.set_ylim(bottom=0)
ax.legend(loc='upper right', ncol=2, fontsize='small')
plt.tight_layout()
plt.show()

As shown in the @fig-peak-amplitudes-part-a, the amplitudes have respectively difference, due to the 

TODO: analyze the difference

# Fit exponential decay to all 10 trials and report mean/std of damping ratio ζ

In [ ]:
zeta_list = []
for idx, fpath in enumerate(files[:10]):
    df = load_trial_csv(fpath)
    t = df['time'].values
    x = df['accel'].values
    x_smooth = signal.savgol_filter(x, 11, 3)
    periods, peaks = estimate_period_via_peaks(t, x_smooth, prominence=np.std(x_smooth)*0.5)
    if isinstance(periods, np.ndarray) and len(peaks) > 1:
        peak_times = t[peaks]
        peak_amps = np.abs(x_smooth[peaks])
        mean_period = np.mean(periods)
        decay = fit_exponential_decay(peak_times, peak_amps, Tn=mean_period)
        zeta_list.append(decay['zeta'])

zeta_arr = np.array(zeta_list)
mean_zeta = np.nanmean(zeta_arr)
std_zeta = np.nanstd(zeta_arr, ddof=1)

print(f"Mean damping ratio ζ: {mean_zeta:.4f} ± {std_zeta:.4f}")

## Discussion


# Part B: Long Pendulum (L ≈ 1.0 m)

Repeat analysis from Part A for `partB_trialXX.csv` files. Adjust `L_long` below if your measured length differs.

## Statistical Analysis

## Damping Analysis

## Discussion

## Comparative Study

In [ ]:
#| label: Part B long Pendulum
L_long = 1.0
file_pattern = os.path.join(DATA_DIR, 'partB_trial*.csv')
files = sorted(glob.glob(file_pattern))
# Copy-paste the same loop for Part B (or refactor into a function). For brevity we reuse code:
results_B = []
for fpath in files:
    df = load_trial_csv(fpath)
    t = df['time'].values
    x = df['accel'].values
    x_smooth = signal.savgol_filter(x, 11, 3)
    periods, peaks = estimate_period_via_peaks(t, x_smooth, prominence=np.std(x_smooth)*0.5)
    if isinstance(periods, np.ndarray):
        mean_period = np.mean(periods)
        std_period = np.std(periods, ddof=1)
    else:
        mean_period = np.nan
        std_period = np.nan
    peak_times = t[peaks] if len(peaks)>0 else np.array([])
    peak_amps = np.abs(x_smooth[peaks]) if len(peaks)>0 else np.array([])
    decay = fit_exponential_decay(peak_times, peak_amps, Tn=mean_period)
    desc = compute_descriptors(x)
    result = {
        'file': os.path.basename(fpath),
        'mean_period': mean_period,
        'std_period': std_period,
        'A0': decay['A0'],
        'k': decay['k'],
        'zeta': decay['zeta'],
        **desc
    }
    results_B.append(result)
pd.DataFrame(results_B)

# Part C: Nonlinear Regime (Large Angles)

Use `partC_trialXX.csv`. For large angle trials (θ0 > 45°), measure period variation vs amplitude and compare with small-angle theory.

## Period Analysis

## Comparison and Discussion


In [ ]:
file_pattern = os.path.join(DATA_DIR, 'partC_trial*.csv')
files = sorted(glob.glob(file_pattern))
period_vs_amp = []
for fpath in files:
    df = load_trial_csv(fpath)
    t = df['time'].values
    x = df['accel'].values
    x_smooth = signal.savgol_filter(x, 11, 3)
    periods, peaks = estimate_period_via_peaks(t, x_smooth, prominence=np.std(x_smooth)*0.5)
    peak_times = t[peaks] if len(peaks)>0 else np.array([])
    peak_amps = np.abs(x_smooth[peaks]) if len(peaks)>0 else np.array([])
    if len(peak_amps)>0 and isinstance(periods, np.ndarray):
        period_vs_amp.append({'file': os.path.basename(fpath), 'mean_period': np.mean(periods), 'mean_amp': np.mean(peak_amps)})
pd.DataFrame(period_vs_amp)

# Part D: Noise Discussion

Visually inspect signals and write your qualitative assessment here. Use the examples shown in previous plots. Answer: What are possible noise sources? How do they affect period/damping estimation?

## Qualitative Noise Observation

### Discussion



# Submission Checklist

- [ ] Export this notebook to PDF: `HW1_{STUDENT_ID}.pdf` (max 20 A4 pages)
- [ ] Create ZIP named `HW1_{STUDENT_ID}.zip` containing:
  - Executed `.ipynb`
  - `Data/` folder with CSVs
  - `Figures/` folder with generated plots
- [ ] Verify all tables are rendered as DataFrames (no screenshots)
- [ ] Confirm all equations and figures are numbered and captioned
- [ ] Ensure reproducibility: all code cells run from top to bottom without manual path edits (except `STUDENT_ID`)


In [ ]:
# Helper: save summary tables and figures
summary_csv = os.path.join(BASE_DIR, f'summary_{STUDENT_ID}.csv')
if 'results_df' in globals():
    results_df.to_csv(os.path.join(BASE_DIR, 'summary_partA.csv'), index=False)
if 'results_B' in globals():
    pd.DataFrame(results_B).to_csv(os.path.join(BASE_DIR, 'summary_partB.csv'), index=False)
print('Saved summary CSVs to', BASE_DIR)

# Reproduce Guide
## How to Reproduce This Analysis

1. **Install Required Software**
    - Install [Python 3.8+](https://www.python.org/downloads/).
    - Install [Quarto](https://quarto.org/docs/get-started/).
    - (Optional) Install [JupyterLab](https://jupyter.org/) for interactive editing.

2. **Set Up the Environment**
    - Open a terminal in the project root folder.
    - (Recommended) Create a virtual environment:
      ```
      python -m venv .venv
      source .venv/bin/activate  # On Windows: .venv\Scripts\activate
      ```
    - Install dependencies:
      ```
      pip install -r requirements.txt
      ```

3. **Prepare Data**
    - Place your raw data CSV files in the `Data/` folder as described in the instructions.
    - Ensure filenames match the expected patterns (e.g., `partA_trial01.csv`).

4. **Edit Metadata**
    - Open `HW1_STUDENTID.qmd` and set your `STUDENT_ID`, name, and date at the top.

5. **Render the Report**
    - To render as HTML:
      ```
      quarto render HW1_STUDENTID.qmd --to html
      ```
    - To render as PDF (requires a LaTeX installation):
      ```
      quarto render HW1_STUDENTID.qmd --to pdf
      ```

6. **Troubleshooting**
    - If you encounter missing package errors, install them with `pip install <package>`.
    - Ensure all code cells run from top to bottom without manual edits except for `STUDENT_ID`.

7. **Export and Submit**
    - Export the rendered PDF and required files as described in the submission checklist.

**Tip:** For interactive exploration, you can open the `.qmd` file in JupyterLab or VS Code with the Quarto extension.

**Contact:** If you have issues reproducing the results, check your Python/Quarto versions and data file structure.